In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import glob

from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import *
import sklearn

/home/kaih/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/kaih/.local/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/kaih/.local/lib/python3.5/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationW

In [2]:
train = pd.read_csv('../../../data/raw/train.csv')
test = pd.read_csv('../../../data/raw/sample_submission_zero.csv')
members = pd.read_csv('../../../data/raw/members.csv')

In [3]:
aggtrain = pd.read_csv('../../results/aggtrain.csv',header=0)

In [4]:
aggtest = pd.read_csv('../../results/aggtest.csv',header=0)

In [5]:
aggtest.shape

(970960, 24)

In [6]:
test.shape

(970960, 2)

In [7]:
# aggtrain =aggtrain.fillna(0)
# aggtest=aggtest.fillna(0)

In [8]:
continuous_features = [
    'bd',
                       'avg(payment_plan_days)',
                       'avg(plan_list_price)',
                       'avg(actual_amount_paid)',
                       'max(num_100)',
                       'max(num_25)',
                       'max(num_75)',
                       'max(num_unq)',
                       'max(num_50)',
                       'max(num_985)',
                       'max(total_secs)',
        'count(transaction_date)', # num transactions
    'count(date)' # num logs
]

categorical_features = [
    'city',
                       'gender',
                       'registered_via',
#                        '_c0',
                       'max(is_cancel)',
                       'min(is_auto_renew)'

]


In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
le = LabelEncoder()
for i in categorical_features:
    aggtrain[i] = le.fit_transform(aggtrain[i].apply(str).values)
    aggtest[i] = le.fit_transform(aggtest[i].apply(str).values)

In [10]:
Xtrain = aggtrain[categorical_features+continuous_features]
Xtest = aggtest[categorical_features+continuous_features]
y = aggtrain.is_churn

In [11]:
from sklearn.metrics import log_loss
def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', log_loss(labels, preds)

In [12]:
fold = 1
round = 0
for i in np.random.randint(1,1000,fold):

    params = {
        'eta': 0.02,
#         'learning_rate': 0.05, # use smaller learning rate to prevent overfit?
        'gamma': 10,
        'max_depth': 7,
        'n_estimators': 200,
        #'random_state': 850564,
#         'min_child_weight': 2,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': i,
        'silent': True,
    }
    x1, x2, y1, y2 = train_test_split(Xtrain, y, test_size=0.33, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 150,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=50)
    if round != 0:
        pred += model.predict(xgb.DMatrix(Xtest), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(Xtest), ntree_limit=model.best_ntree_limit)
    round += 1
pred /= fold


[0]	train-log_loss:0.676522	valid-log_loss:0.676525
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-log_loss:0.280326	valid-log_loss:0.280625
[100]	train-log_loss:0.17681	valid-log_loss:0.177481


In [13]:
xgbprediction=aggtest[['msno','is_churn']]
xgbprediction['is_churn'] = pred.clip(0.0000001, 0.999999)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
xgbprediction.to_csv('../../results/xgbsubmission.csv')

In [19]:
aggtrain_enc = aggtrain[categorical_features+continuous_features+['is_churn']]
aggtest_enc = aggtest[categorical_features+continuous_features+['is_churn']]
aggtrain_enc =aggtrain_enc.fillna(-1)
aggtest_enc=aggtest_enc.fillna(-1)
for i in categorical_features:
    aggtrain_enc[i] = aggtrain[i].astype('category')
    aggtest_enc[i] = aggtest[i].astype('category')

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
aggtrain_enc = pd.get_dummies(aggtrain_enc)
aggtest_enc = pd.get_dummies(aggtest_enc)

Xtrain_enc = aggtrain_enc.loc[:, aggtrain_enc.columns != 'is_churn']
Xtest_enc = aggtest_enc.loc[:, aggtest_enc.columns != 'is_churn']
# y = np.asarray(aggtrain_enc['is_churn'], dtype="float64")
y = aggtrain.is_churn

#### Linear Regression

In [195]:
Xtest_enc=Xtest_enc.loc[:, Xtest_enc.columns != 'max(is_cancel)_2']

In [196]:
Xtest_enc=Xtest_enc.loc[:, Xtest_enc.columns != 'min(is_auto_renew)_2']

In [199]:
lr = LinearRegression()
lr.fit(Xtrain_enc,y)
lrpred = lr.predict(Xtest_enc)


In [201]:
lrprediction=aggtest[['msno','is_churn']]
lrprediction['is_churn'] = lrpred.clip(0.0000001, 0.999999)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Random Forest

In [202]:
rf = RandomForestRegressor(n_estimators=100,n_jobs=-1)

In [ ]:
rf.fit(Xtrain_enc,y)
rfpred=rf.predict(Xtest_enc)

In [217]:
rfprediction=aggtest[['msno','is_churn']]
rfprediction['is_churn'] = rfpred.clip(0.0000001, 0.999999)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
rfprediction.to_csv('../../results/rfsubmission.csv')

In [245]:
prediction = xgbprediction.merge(rfprediction,on='msno')
# prediction['is_churn'] = (prediction['is_churn_x']+prediction['is_churn_y']+prediction['is_churn'])/3
prediction['is_churn'] = (prediction['is_churn_x']+prediction['is_churn_y'])/2

In [246]:
prediction = prediction[['msno','is_churn']]

In [247]:
prediction.to_csv('../../results/rfxgb.csv')

In [248]:
prediction = xgbprediction.merge(rfprediction,on='msno')
prediction = prediction.merge(lrprediction,on='msno')
prediction['is_churn'] = (prediction['is_churn_x']+prediction['is_churn_y']+prediction['is_churn'])/3
prediction = prediction[['msno','is_churn']]
prediction.to_csv('../../results/lrrfxgb.csv')

#### SVR (no parallelism.. very slow)

In [ ]:
# from sklearn.svm import SVR

# svr = SVR()
# svr.fit(Xtrain_enc,y)
# rfpred=svr.predict(Xtest_enc)


very.... ssssllloooooooooooowww...... like, I've been running it for over 40 hours and it just won't stop kinda slow.. don't try this. Training complexity of nonlinear SVM is generally between O(n^2) and O(n^3).

### load trained randomforest 

In [17]:
import pickle
filename = '../../models/rf_depth5nest100leaf1.sav'
rf_load = pickle.load(open(filename, 'rb'))

In [ ]:
rfpred=rf_load.predict(Xtest_enc)